In [2]:
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numba import jit
import time

In [2]:
fitness_per_eval = []
fitness_per_iter = []

In [3]:
rosenbrockdf = pd.read_csv('../data/rosenbrock.csv')
f_biasdf = pd.read_csv('../data/f_bias.csv')

rosenbrock = rosenbrockdf.fvalue.values
f_bias = f_biasdf.fvalue.values

In [4]:
f_xstar = f_bias[2]
search_space = (-100,100)

double Shifted_Rosenbrock( int dim , double* x ){
    
    int i;
    double z[dim];
    double F = 0;

    for(i=0;i<dim;i++) z[i] = x[i] - rosenbrock[i] + 1;   

    for(i=0;i<dim-1;i++){    
        F = F + 100*( pow((pow(z[i],2)-z[i+1]) , 2) ) + pow((z[i]-1) , 2);
    }
    return F + f_bias[2]; 
}

In [3]:
class ShiftedRosen(FloatProblem):
    def __init__(self, number_of_variables: int = 10):
        super(MyRosen, self).__init__()
        self.number_of_objectives = 1
        self.number_of_variables = number_of_variables
        self.number_of_constraints = 0

        self.obj_directions = [self.MINIMIZE]
        self.obj_labels = ['f(x)']

        self.lower_bound = [-100. for _ in range(number_of_variables)]
        self.upper_bound = [100. for _ in range(number_of_variables)]

        FloatSolution.lower_bound = self.lower_bound
        FloatSolution.upper_bound = self.upper_bound

    def evaluate(self, solution: FloatSolution) -> FloatSolution:
        dim = solution.number_of_variables
        F = 0
        z = np.empty(50)
        x = solution.variables
        for i in range(dim - 1):
            z[i] = x[i] - rosenbrock[i] + 1

        for i in range(dim - 2):
            F += 100 * ((z[i] ** 2 - z[i + 1]) ** 2) + (z[i] - 1) ** 2

        result = F + f_xstar

        solution.objectives[0] = result
        return solution

    def get_name(self) -> str:
        return 'Shifted RosenBrock'